# Проект для «Викишоп»

In [135]:
import warnings
warnings.filterwarnings("ignore")
import time
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from langdetect import detect
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import (train_test_split, 
                                     RandomizedSearchCV)
from sklearn.metrics import f1_score

# Импортирую библиотеку для отображения статус-бара
from tqdm import tqdm

In [136]:
STATE = np.random.RandomState(12345)

# Получение данных

In [137]:
try:
    data = pd.read_csv('C:/0/d/toxic_comments.csv')
    print('Прочитали данные с диска')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')
    print('Прочитали данные в сети')

Прочитали данные с диска


Ознакомимся с данными

In [138]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Установим индекс

In [140]:
data.rename(columns = {'Unnamed: 0':'id'}, inplace = True )

In [141]:
data.sort_values(by='id',inplace=True)

In [142]:
if len(data.id) != len(data.id.unique()):
    print('Значения индекса повторяются')
else: print('Значения индекса уникальны')

Значения индекса уникальны


In [143]:
data = data.set_index('id')

Переведем все тексты в нижний регистр (поскольку словарь стоп-слов работает с нижним регистром)

In [144]:
data['text'] = data['text'].str.lower()

In [145]:
data.head()

,text,toxic
id,,
0,explanation\nwhy the edits made under my usern...,0
1,d'aww! he matches this background colour i'm s...,0
2,"hey man, i'm really not trying to edit war. it...",0
3,"""\nmore\ni can't make any real suggestions on ...",0
4,"you, sir, are my hero. any chance you remember...",0


## Подготовка

### Подвыборка
Поскольку датасет большой для отработки алгоритма создам функцию для подвыборки, для отработки методов.

In [146]:
def get_sample(data, fraction):
    """
    Получение случайной подвыборки из исходной выборки.
    :param data: исходная выборка
    :param fraction: доля (от 0 до 1), которую отбираем из исходной выборки
    :return: подвыборка
    """
    return data.sample(frac=fraction, random_state=STATE)

Далее беру всю выборку целиком.

In [147]:
sample = get_sample(data, 1)

### Проверка языков датасета

Оценим долю языков по датасету. Функцию ниже закомментировал. Она рабочая, и понадобилась только один раз для проверки данных.

In [148]:
# def detect_languages(data):
#     num_english, num_russian, num_other = 0, 0, 0
#     for text in tqdm(data['text']):
#         try:
#             lang = detect(text)
#         except:
#             lang = 'unknown'
#         if lang == 'en':
#             num_english += 1
#         elif lang == 'ru':
#             num_russian += 1
#         else:
#             num_other += 1
#     total = num_english + num_russian + num_other
#     print(f"English: {num_english/total:.2%}, Russian: {num_russian/total:.2%}, Other: {num_other/total:.2%}")

Примерное время работы данной функции на моем компьютере для полного датасета составило 15 минут. Также сам результат функции нигде не используется, поэтому, я закомментировал блок ниже.

In [149]:
# detect_languages(data)
# detect_languages(sample)

Доля английского на всем датасете составляет почти 97%. Поэтому для лемматизации буду использовать nltk без русского словаря

### Токенизация

In [150]:
"""
загрузим набор данных, содержащий предобученные модели токенизации для разных языков
"""
nltk.download('punkt')

def tokenize_text(text):
    """
    Токенизация текста на отдельные слова.
    :param text: текст для токенизации
    :return: список токенов
    """
    tokens = nltk.word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [151]:
# токенизируем текст. время на весь датасет около 2 минут
sample['tokens'] = [tokenize_text(text) for text in tqdm(sample['text'])]

100%|██████████| 159292/159292 [01:29<00:00, 1771.85it/s]


In [152]:
sample['tokens']

id
109583    [expert, categorizers, why, is, there, no, men...
105077                      [``, noise, fart, *, talk., ``]
82244     [an, indefinite, block, is, appropriate, ,, ev...
18740     [i, do, n't, understand, why, we, have, a, scr...
128310    [hello, !, some, of, the, people, ,, places, o...
                                ...                        
110090    [hahaha, ., ), i, dont, live, in, a, lie, like...
85493             [march, 2006, –, march, 2006, ], ], |, }]
133387    [``, agreed, ., we, really, should, try, to, s...
130469    [``, umm, killer, do, you, not, like, that, he...
77361     [bradford, city, i, am, removing, unreferanced...
Name: tokens, Length: 159292, dtype: object

### Удаление нетекстовых символов

In [153]:
def remove_nonletters(tokens):
    """
    Удаление символов, кроме букв, из списка токенов.
    :param tokens: список токенов для обработки
    :return: список токенов с удаленными символами, кроме букв
    """
    pattern = r'[^a-zA-Zа-яА-Я\s]'
    # удаление символов в каждом токене
    filtered_tokens = [re.sub(pattern, '', token) for token in tokens]
    return filtered_tokens

In [154]:
# время - около 15 сек
sample['tokens'] = [remove_nonletters(text) for text in tqdm(sample['tokens'])]

100%|██████████| 159292/159292 [00:12<00:00, 12563.84it/s]


In [155]:
sample['tokens']

id
109583    [expert, categorizers, why, is, there, no, men...
105077                            [, noise, fart, , talk, ]
82244     [an, indefinite, block, is, appropriate, , eve...
18740     [i, do, nt, understand, why, we, have, a, scre...
128310    [hello, , some, of, the, people, , places, or,...
                                ...                        
110090    [hahaha, , , i, dont, live, in, a, lie, like, ...
85493                          [march, , , march, , , , , ]
133387    [, agreed, , we, really, should, try, to, stic...
130469    [, umm, killer, do, you, not, like, that, he, ...
77361     [bradford, city, i, am, removing, unreferanced...
Name: tokens, Length: 159292, dtype: object

### Лемматизация

In [156]:
# Загрузка WPOS-теггера и словаря
nltk.download('averaged_perceptron_tagger')
# Загрузка английского словаря с лексикой
nltk.download('wordnet')
# Загрузка русского словаря с лексикой
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [157]:
def get_wordnet_pos(tag):
    """
    Преобразование POS-тегов из формата, используемого в nltk.pos_tag(), в формат,
    используемый в WordNetLemmatizer.
    :param tag: POS-тег из nltk.pos_tag()
    :return: POS-тег для WordNetLemmatizer
    """
    if tag.startswith('J'):
        return 'a'  # прилагательное
    elif tag.startswith('V'):
        return 'v'  # глагол
    elif tag.startswith('N'):
        return 'n'  # существительное
    elif tag.startswith('R'):
        return 'r'  # наречие
    else:
        return None

In [159]:
def lemmatize_with_pos(tokens):
    """
    Лемматизация токенов с указанием части речи.
    :param tokens: список токенов
    :return: список лемматизированных токенов
    """
    # Получение POS-тегов для каждого токена
    pos_tags = nltk.pos_tag(tokens)
    # Преобразование POS-тегов к формату, который используется в WordNetLemmatizer
    pos_tags = [(tag[0], get_wordnet_pos(tag[1])) for tag in pos_tags]
    # Лемматизация токенов с учетом их части речи
    lemmatized_tokens = []
    for tag in pos_tags:
        if tag[1] is not None:
            lemma = lemmatizer.lemmatize(tag[0], tag[1])
            lemmatized_tokens.append(lemma)
    return lemmatized_tokens

In [160]:
# время на всем датафрейме - около 14 минут
sample['lemms'] = [lemmatize_with_pos(text) for text in tqdm(sample['tokens'])]

100%|██████████| 159292/159292 [13:46<00:00, 192.64it/s]


Сравним, сколько по времени выполняется apply

In [184]:
# можно не запускать - это тест - для сравнения скорости
# %time
# start = time.time()
# sample['lemms_prog_applay'] = sample['tokens'].apply(lemmatize_with_pos)
# end = time.time()
# l_prog_apply_time = end-start

Wall time: 0 ns


In [186]:
# l_prog_apply_time/60

12.7402152578036

In [195]:
sample

,text,toxic,tokens,lemms,lemms_prog_applay
id,,,,,
109583,expert categorizers \n\nwhy is there no menti...,0,"[expert, categorizers, why, is, there, no, men...","[expert, categorizers, be, there, mention, fac...","[expert, categorizers, be, there, mention, fac..."
105077,"""\n\n noise \n\nfart* talk. """,1,"[, noise, fart, , talk, ]","[, noise, fart, , talk, ]","[, noise, fart, , talk, ]"
82244,"an indefinite block is appropriate, even for a...",0,"[an, indefinite, block, is, appropriate, , eve...","[indefinite, block, be, appropriate, , even, m...","[indefinite, block, be, appropriate, , even, m..."
18740,i don't understand why we have a screenshot of...,0,"[i, do, nt, understand, why, we, have, a, scre...","[i, do, understand, have, screenshot, ap, s, g...","[i, do, understand, have, screenshot, ap, s, g..."
128310,"hello! some of the people, places or things yo...",0,"[hello, , some, of, the, people, , places, or,...","[hello, , people, , place, thing, have, write,...","[hello, , people, , place, thing, have, write,..."
...,...,...,...,...,...
110090,hahaha. ) i dont live in a lie like you and do...,1,"[hahaha, , , i, dont, live, in, a, lie, like, ...","[hahaha, , , i, dont, live, lie, dont, deny, t...","[hahaha, , , i, dont, live, lie, dont, deny, t..."
85493,march 2006 – march 2006]]\n \n\n|},0,"[march, , , march, , , , , ]","[march, , , march, , , , , ]","[march, , , march, , , , , ]"
133387,"""\n\nagreed. we really should try to stick to...",0,"[, agreed, , we, really, should, try, to, stic...","[, agree, really, try, stick, subject, article...","[, agree, really, try, stick, subject, article..."


### Удаление стоп-слов

In [196]:
# загрузим словарь стоп-слов
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [197]:
stop_words = set(stopwords.words('english'))

In [198]:
def remove_stopwords(tokens):
    """
    Удаление стоп-слов из списка токенов.
    :param tokens: список токенов для обработки
    :return: список токенов с удаленными стоп-словами
    """
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [201]:
sample['clear'] = [remove_stopwords(text) for text in tqdm(sample['lemms'])]

100%|██████████| 159292/159292 [00:01<00:00, 112998.47it/s]


### Объединение токенов

In [202]:
def join_tokens(tokens):
    return ' '.join(tokens) 

In [203]:
sample['clear'] = [join_tokens(text) for text in tqdm(sample['clear'])]

100%|██████████| 159292/159292 [00:00<00:00, 377433.64it/s]


In [204]:
sample.head()

,text,toxic,tokens,lemms,lemms_prog_applay,clear
id,,,,,,
109583,expert categorizers \n\nwhy is there no menti...,0,"[expert, categorizers, why, is, there, no, men...","[expert, categorizers, be, there, mention, fac...","[expert, categorizers, be, there, mention, fac...",expert categorizers mention fact nazis particu...
105077,"""\n\n noise \n\nfart* talk. """,1,"[, noise, fart, , talk, ]","[, noise, fart, , talk, ]","[, noise, fart, , talk, ]",noise fart talk
82244,"an indefinite block is appropriate, even for a...",0,"[an, indefinite, block, is, appropriate, , eve...","[indefinite, block, be, appropriate, , even, m...","[indefinite, block, be, appropriate, , even, m...",indefinite block appropriate even minor infra...
18740,i don't understand why we have a screenshot of...,0,"[i, do, nt, understand, why, we, have, a, scre...","[i, do, understand, have, screenshot, ap, s, g...","[i, do, understand, have, screenshot, ap, s, g...",understand screenshot ap gui ub someone remedy
128310,"hello! some of the people, places or things yo...",0,"[hello, , some, of, the, people, , places, or,...","[hello, , people, , place, thing, have, write,...","[hello, , people, , place, thing, have, write,...",hello people place thing write article tryfo...


### Векторизация

Поскольку обучать модели буду с помощью рандомсерча, векторизацию буду делать для всех фолдов внутри рандомсерча. Поэтому пока просто разделю выборки на обучающую и тестовую

In [205]:
X_train, X_test, y_train, y_test = \
train_test_split(sample['clear'], sample['toxic'], test_size=0.2, random_state=STATE)

## Обучение

Буду обучать 2 модели. Создам датафрейм для записи итоговых данных по моделям

In [206]:
models = ['LogReg','Light GBM']
model_results = pd.DataFrame(columns=['f1_train','time_train','f1_test','time_test'],
                             index=models,dtype=float)

Создам функцию для рандомного поиска гиперпараметров

In [207]:
def f_random_cv(model,grid,CV,score,features,target):
    # Определяем пайплайн
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('model', model)
    ])
    
    random_cv = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=grid,
        cv=CV, 
        scoring = score,
        n_jobs = -1,
        random_state=STATE)
    random_cv.fit(features, target)
    return random_cv

In [208]:
# для всех моделей
scoring = 'f1'
CV = 4

### Логистическая регрессия

In [210]:
# Определяем модель
# model_logr = LogisticRegression(class_weight='balanced')
model_logr = LogisticRegression(random_state=STATE)
# определим пустую сетку
# grid = {}
# Определяем диапазон параметров
grid = {
    'model__penalty': ['l1', 'l2'], 
    'model__C': [0.01, 0.1, 1, 10, 100, 1000],
#     'model__fit_intercept': [True, False],
#     'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
#     'model__max_iter': [50, 100, 200, 500, 1000],
#     'vectorizer__max_df': [0.8, 0.9],
#     'vectorizer__min_df': [0.01, 0.05],
#     'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)]
}

In [211]:
# Подбираем лучшие гиперпараметры - расчет на полном датасете с пустой сеткой ГП- около 30 сек
# расчет на полном датасете для сетки ГП
#         {'penalty': ['l1', 'l2'],'C': [0.01, 0.1, 1, 10, 100, 1000]} - 2 минуты
for i in tqdm(range(1)): # применил цикл с одной итерацией для запуска статус бара tqdm
    random_cv_logr = f_random_cv(model_logr,grid,CV,scoring,X_train,y_train)

100%|██████████| 1/1 [02:04<00:00, 124.76s/it]


In [212]:
# лучшие гиперпараметры и значение метрики
print("Best params: ", random_cv_logr.best_params_)
print("Best score: ", random_cv_logr.best_score_)

Best params:  {'model__penalty': 'l2', 'model__C': 10}
Best score:  0.7672320284571663


In [214]:
mean_fit_time = random_cv_logr.cv_results_['mean_fit_time']
total_time = np.sum(mean_fit_time)

model_results.loc['LogReg',['f1_train','time_train','f1_test','time_test']]=\
[random_cv_logr.best_score_,total_time,'n/a','n/a']

### Light GBM

In [215]:
# инициализируем модель
model_lgb = lgb.LGBMClassifier(random_state=STATE)

In [222]:
# установим сетку гиперпараметров
# на пустой сетке
# grid = {}
grid = {
    'model__num_leaves': [10, 20, 30, 40, 50]
    ,'model__learning_rate': [0.01,0.05, 0.1, 0.5]
    ,'model__max_depth': [-1 , 5, 10, 20]
}

In [223]:
# Обучим модель полном датасете
# время расчета на пустой сетке - 2 менуты
# время расчета на непустой сетке- 10 минут
for i in tqdm(range(1)): # применил цикл с одной итерацией для запуска статус бара tqdm
    random_cv_lgbm = f_random_cv(model_lgb,grid,CV,scoring,X_train,y_train)

100%|██████████| 1/1 [10:14<00:00, 614.90s/it]


In [224]:
# Выводим лучшие гиперпараметры и значение метрики
print("Best params: ", random_cv_lgbm.best_params_)
print("Best score: ", random_cv_lgbm.best_score_)

Best params:  {'model__num_leaves': 40, 'model__max_depth': 20, 'model__learning_rate': 0.5}
Best score:  0.7564948961711593


In [225]:
mean_fit_time = random_cv_lgbm.cv_results_['mean_fit_time']
total_time = np.sum(mean_fit_time)

model_results.loc['Light GBM',['f1_train','time_train','f1_test','time_test']]=\
[random_cv_lgbm.best_score_,total_time,'n/a','n/a']

## Тестирование

Лучшей моделью является логистическая регрессия. Тем не менее выведу результаты тестирования для двух моделей.

In [226]:
best_logr = random_cv_logr.best_estimator_
start = time.time()
pred_logr = best_logr.predict(X_test)
end = time.time()
best_logr_pred_time = end-start
best_logr_f1 = f1_score(y_test, pred_logr)
model_results.loc['LogReg',['f1_test','time_test']]=\
[round(best_logr_f1,5),round(best_logr_pred_time,6)]
# model_results.round(3)

In [227]:
best_lgbm = random_cv_lgbm.best_estimator_
start = time.time()
pred_lgbm = best_lgbm.predict(X_test)
end = time.time()
best_lgbm_pred_time = end-start
best_lgbm_f1 = f1_score(y_test, pred_lgbm)
model_results.loc['Light GBM',['f1_test','time_test']]=\
[round(best_lgbm_f1,5),round(best_lgbm_pred_time,6)]
model_results.round(5)

,f1_train,time_train,f1_test,time_test
LogReg,0.76723,98.32813,0.77199,1.05985
Light GBM,0.75649,554.22105,0.76399,1.284271


## Выводы

Логистическая регрессия оказалась наиболее эффективной моделью для данной задачи, но необходимо отметить, что LGBM на тесте незначительно улучшила метрику.

В целом могу сказать, что обработка тектсов - это самая ресурсоемкая (из того, чем приходилось мне заниматься) сфера ДС. Возможны улучшения показателей при другой обработке признаков (другие библиотеки, другой подход к тэгам и т.д.) Но в виду ресурсоемкости проводить такие эксперименты очень накладно (по крайней мере я бы не стал это делать, просто пробуя разные инструменты- как это делал с табличными данными).